# Everruns Agent API Example

This notebook demonstrates how to use the Everruns API to:
1. Create an agent with a system prompt
2. Create a session (conversation)
3. Send a message to trigger the agentic loop
4. Read events until the assistant response appears
5. Continue the conversation

## Prerequisites

- Everruns API server running at `http://localhost:9000`
- Python packages: `requests`

In [ ]:
# Install required packages
!pip install requests

In [ ]:
import requests
import json
import time

# Configuration
BASE_URL = "http://localhost:9000"
API_V1 = f"{BASE_URL}/v1"

def print_json(data):
    """Pretty print JSON data."""
    print(json.dumps(data, indent=2, default=str))

## 1. Health Check

Verify the API server is running.

In [ ]:
response = requests.get(f"{BASE_URL}/health")
response.raise_for_status()
print("API Status:", response.json())

## 2. Create an Agent

An agent is a configuration for an agentic loop with a system prompt.

In [ ]:
agent_data = {
    "name": "Python Example Agent",
    "description": "A helpful assistant created via the Python API",
    "system_prompt": "You are a helpful assistant. Be concise and friendly.",
    "tags": ["example", "python"]
}

response = requests.post(f"{API_V1}/agents", json=agent_data)
response.raise_for_status()
agent = response.json()

agent_id = agent["id"]
print(f"Created agent: {agent_id}")

## 3. Create a Session

A session is an instance of conversation with the agent.

In [ ]:
session_data = {
    "title": "Python API Test Session"
}

response = requests.post(f"{API_V1}/agents/{agent_id}/sessions", json=session_data)
response.raise_for_status()
session = response.json()

session_id = session["id"]
print(f"Created session: {session_id}")

## 4. Send a Message

Sending a user message triggers the agentic loop workflow.

In [ ]:
message_data = {
    "message": {
        "role": "user",
        "content": [
            {"type": "text", "text": "What is 2 + 2? Please explain briefly."}
        ]
    }
}

response = requests.post(
    f"{API_V1}/agents/{agent_id}/sessions/{session_id}/messages",
    json=message_data
)
response.raise_for_status()
print(f"Sent message: {response.json()['id']}")

## 5. Read Events

Poll the events endpoint until the assistant's response appears.

In [ ]:
def read_until_output(agent_id: str, session_id: str, timeout: int = 60):
    """Poll events until an assistant message appears.
    
    Returns the assistant's response text.
    """
    url = f"{API_V1}/agents/{agent_id}/sessions/{session_id}/events"
    seen_sequences = set()
    start_time = time.time()
    
    while time.time() - start_time < timeout:
        response = requests.get(url)
        response.raise_for_status()
        events = response.json().get("data", [])
        
        for event in events:
            seq = event["sequence"]
            if seq in seen_sequences:
                continue
            seen_sequences.add(seq)
            
            event_type = event["event_type"]
            data = event["data"]
            
            # Print event details
            if event_type == "message.assistant":
                message = data.get("message", {})
                content = message.get("content", [])
                text_parts = [p["text"] for p in content if p.get("type") == "text"]
                text = "\n".join(text_parts)
                print(f"[{event_type}] {text[:100]}{'...' if len(text) > 100 else ''}")
                return text
            elif event_type == "message.tool_call":
                message = data.get("message", {})
                content = message.get("content", [])
                for part in content:
                    if part.get("type") == "tool_call":
                        print(f"[{event_type}] {part.get('name')}({part.get('arguments')})")
            elif event_type == "message.tool_result":
                message = data.get("message", {})
                content = message.get("content", [])
                for part in content:
                    if part.get("type") == "tool_result":
                        result = part.get("result", "")
                        print(f"[{event_type}] {str(result)[:80]}...")
            elif event_type == "LoopCompleted":
                print(f"[{event_type}] iterations={data.get('total_iterations')}")
            elif event_type in ("LoopStarted", "IterationStarted", "LlmCallStarted", "LlmCallCompleted"):
                print(f"[{event_type}]")
            elif event_type == "LoopError":
                print(f"[{event_type}] {data.get('error')}")
                return None
        
        time.sleep(0.5)
    
    return None

# Read events until we get the assistant's response
print("Waiting for assistant response...\n")
assistant_response = read_until_output(agent_id, session_id)

if assistant_response:
    print(f"\n--- ASSISTANT ---")
    print(assistant_response)
else:
    print("No response received")

## 6. Continue the Conversation

Send another message and read the response.

In [ ]:
# Send follow-up message
message_data = {
    "message": {
        "content": [
            {"type": "text", "text": "What about 3 + 3?"}
        ]
    }
}

response = requests.post(
    f"{API_V1}/agents/{agent_id}/sessions/{session_id}/messages",
    json=message_data
)
response.raise_for_status()
print("Sent follow-up message\n")

# Read response
assistant_response = read_until_output(agent_id, session_id)

if assistant_response:
    print(f"\n--- ASSISTANT ---")
    print(assistant_response)

## 7. Cleanup

Delete the session and archive the agent.

In [ ]:
# Delete session
requests.delete(f"{API_V1}/agents/{agent_id}/sessions/{session_id}")
print(f"Deleted session: {session_id}")

# Archive agent
requests.delete(f"{API_V1}/agents/{agent_id}")
print(f"Archived agent: {agent_id}")

## Summary

Core API workflow:

1. `POST /v1/agents` - Create agent
2. `POST /v1/agents/{id}/sessions` - Create session
3. `POST /v1/agents/{id}/sessions/{id}/messages` - Send message
4. `GET /v1/agents/{id}/sessions/{id}/events` - List events (JSON)

API docs: http://localhost:9000/swagger-ui/